In [ ]:
"""
This notebook generates all data found in the paper.
On a newish laptop, it should run in ~30 min.
"""

%load_ext autoreload
%autoreload 2
from jax import config
config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np
import pickle


import models

import matplotlib.pyplot as plt


#This ignores a FutureWarning due to diffrax doing something that will lead to an error in future JAX releases
#You may want to comment it out to check that nothing else is going wrong.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def fn(n):
    return "../data/fig_"+str(n)+".pkl"


g = 432
I0 = 5
S_E = 1.44
S_I = 0.72
m0 = 864*10

common_params = dict(m_0 = m0, M=0, S_E=S_E, S_I=S_I, I_0=I0, γ=g, f_K=np.sqrt(20000), f_n=2)
uncommon_params = {}
uncommon_params["A"] = dict(k_r=1/24, g_K=4000, g_n=2, g_k=116)
uncommon_params["B"] = dict(k_r=1/24, g_K=4000, g_n=2, g_k=1700)
uncommon_params["C"] = dict(k_r=1/36, g_K=4000, g_n=2, g_k=1700*1.5)


#set beta_tot to give fasting G of 110 before the emergency
for key in ["A", "B", "C"]:
    params = dict(G_0=110.0, **common_params, **uncommon_params[key] )
    B0 = g*models.adaptive_beta(params)
    uncommon_params[key]["beta_tot"] = B0
        

small_meal_rate = 300.0
large_meal_rate = 8500.0
insulin_rate = 70.0*g/100 

def fig_2_data(params_A, params_B, params_C, params_D, sp_A, sp_B, sp_C, sp_D):
    g_A, _, beta_A, _, _, _ = models.full_simulation(params_A,**sp_A)
    g_B, _, beta_B, _, _, _ = models.full_simulation(params_B, **sp_B)
    g_C, _, beta_C, _, _, _ = models.full_simulation(params_C, **sp_C)
    g_D, _, beta_D, _, _, t = models.full_simulation(params_D, **sp_D)
    
    with open(fn(2), "wb") as f:
        pickle.dump(dict(t=t, A=dict(G=g_A, β=beta_A, sp=sp_A), B=dict(G=g_B, β=beta_B, sp=sp_B), C=dict(G=g_C, β=beta_C, sp=sp_C), D=dict(G=g_D, β=beta_D, sp=sp_D)), f)
        
def fig_5_data(params_A, params_B):
    B0_B = params_B['beta_tot']

    data = {}
    data['A'] = models.SI_beta_scan(params_A, SI_range=[0.1, 3.0], beta_range=[0, 3*B0_B], SI_samples=100,beta_samples=100, G_0=80,a=0.0)
    data['B'] = models.SI_beta_scan(params_B, SI_range=[0.1, 3.0], beta_range=[0, 3*B0_B], SI_samples=100,beta_samples=100, G_0=80,a=0.0,table_beta_samples=6000)

    with open(fn(5), "wb") as f:
        pickle.dump(data, f)


def fig_6_data(params_B, sp_B, G_min):
    g_0, I_0, beta_0, _, F_0, t = models.full_simulation(params_B, **sp_B)


    sp_prime = sp_B | {"insulin": models.fasting_maximum_insulin(G_min)}
    g_1, I_1, beta_1, _, F_1, _ = models.full_simulation(params_B, **sp_prime)

    with open(fn(6), "wb") as f:
        pickle.dump(dict(t=t, A=dict(G=g_0, I = I_0, β=beta_0, F=F_0, sp=sp_B), B=dict(G=g_1, I = I_1, β=beta_1, F=F_1, sp=sp_B)), f)


def compute_M(params):
    def computer(M):
        G, f = models.all_eq(params | {"M":M })
        if len(G) > 1:
            return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
        else:
            return np.array([G[0], -116, -116, f[0], -116, -116])
    return computer

def compute_SI(params):
    def computer(SI):
        G, f = models.all_eq(params | {"S_I":SI })
        if len(G) > 1:
            return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
        else:
            return np.array([G[0], -116, -116, f[0], -116, -116])
    return computer

def compute_B(params):
    def computer(B):
        G, f = models.all_eq(params | {"β":params['β']*B })
        if len(G) > 1:
            return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
        else:
            return np.array([G[0], -116, -116, f[0], -116, -116])

    return computer

def compute_F(params):
    def computer(F):
        G, f = models.all_eq(params | {"F_I": F})
        if len(G) > 1:
            return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
        else:
            return np.array([G[0], -116, -116, f[0], -116, -116])

    return computer

def compute_gk(params):
    def computer(k):
        G, f = models.all_eq(params | {"g_k": k})
        if len(G) > 1:
            return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
        else:
            return np.array([G[0], -116, -116, f[0], -116, -116])

    return computer

def fixed_step_bisect(compute_func, bisect_func, x0, max_x, delta):
    X = []
    Y = []
    x = x0
    prev_y = compute_func(x)
    X.append(x)
    Y.append(prev_y)
    x = x0+delta
    y = compute_func(x)
    while (not bisect_func(y, prev_y)):
        X.append(x)
        Y.append(y)
        prev_y = y
        x = x+delta
        y = compute_func(x)
        if x >= max_x:
            return X, Y

    return X, Y

def ramp_up_step(compute_func, x0, delta, depth):
    X = []
    Y = []
    x = x0
    prev_y = compute_func(x)
    X.append(x)
    Y.append(prev_y)
    for i in range(depth):
        x = x0+ delta*(2**i)
        y = compute_func(x)
        X.append(x)
        Y.append(y)
    return X,Y
        

def bisecting_sweep(compute_func, bisect_func, x0, delta, max_x, max_depth, snap=True, symmetric=True):

    assert snap
    X = []
    Y = []
    x = x0
    depth = 0
    prev_y = compute_func(x)
    X.append(x)
    Y.append(prev_y)
    x = x0+delta
    
    while True:
        temp_X, temp_Y = fixed_step_bisect(compute_func, bisect_func, x, max_x, delta / 2**depth)

        X = X + temp_X[1:]
        Y = Y + temp_Y[1:]

        if X[-1] >= max_x:
            return X, Y

        elif depth < max_depth:
            depth += 1
            x = X[-1]

        else: #snap to grid over the bisection point 
            if symmetric:
                temp_X, temp_Y = ramp_up_step(compute_func, x, delta / 2**depth, depth)
                X = X + temp_X[1:]
                Y = Y + temp_Y[1:]
            depth = 0
            x = delta*(X[-1]//delta) + delta

    return X, Y

def compute_B_M(M):
    G, f = models.all_eq(params_B | {"M":M })
    if len(G) > 1:
        return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
    else:
        return np.array([G[0], -116, -116, f[0], -116, -116])

def compute_B_SI(S_I):
    G, f = models.all_eq(params_B | {"S_I":S_I })
    if len(G) > 1:
        return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
    else:
        return np.array([G[0], -116, -116, f[0], -116, -116])

def compute_B_B(B):
    G, f = models.all_eq(params_B | {"β":params_B['β']*B })
    if len(G) > 1:
        return np.array([G[0], G[1], G[2], f[0], f[1], f[2]])
    else:
        return np.array([G[0], -116, -116, f[0], -116, -116])



def bisect(prev_y, y):
    return not np.sum(y > 0) == np.sum(prev_y > 0)


def total_flux(params, G, alt_mode=False):
    f = models.eq_beta_frac(params, G)
    I = f*models.evaluate(models.get_f(params), G)

    if alt_mode:
        return (params["S_E"] + params["S_I"]*I)*G
    else:
        return params["M"] + params["m_0"]/(I+params["I_0"])

def process(params, M_list, G_list, β_list, flux=False):
    arg = np.where(G_list > 0)[0]
    m = np.zeros(len(arg))

    for i, j in enumerate(arg):
        if flux:
            m[i] = total_flux(params | {"M": M_list[j]}, G_list[j])
        else:
            m[i] = M_list[j]
    return m, G_list[arg], β_list[arg]


def bifurcation_data(params, compute_func, min_param, param_spacing, max_param, param_name, key):
        input_list, blah = bisecting_sweep(compute_func(params), bisect, min_param, param_spacing, max_param, 32, snap=True)
        input_list = np.array(input_list)
        G_l = np.transpose(blah)[0]
        G_u = np.transpose(blah)[1]
        G_h = np.transpose(blah)[2]
        beta_l = np.transpose(blah)[3]
        beta_u = np.transpose(blah)[4]
        beta_h = np.transpose(blah)[5]
        if not key == "A":
            arg = np.where(G_l>np.max(G_h))
            G_h[arg] = G_l[arg]
            G_l[arg] = -116
            beta_h[arg] = beta_l[arg]
            beta_l[arg] = -116

        
    
        cut_l = np.where(G_l >= 80)
        cut_u = np.where(G_u >= 80)
        cut_h = np.where(G_h >= 80)

        return { param_name+"_list_l": input_list[cut_l], "G_l": G_l[cut_l], "beta_l": beta_l[cut_l],
                 param_name+"_list_u": input_list[cut_u], "G_u": G_u[cut_u], "beta_u": beta_u[cut_u], 
                 param_name+"_list_h": input_list[cut_h], "G_h": G_h[cut_h], "beta_h": beta_h[cut_h] }
              

def fig_M_bif_data(params_A, params_B):

    output = {}
    for key, params in zip(["A", "B"], [params_A, params_B]):
        output[key] = bifurcation_data(params, compute_M, -1000, 1, 1000, "M", key)    

    with open(fn("M_bif"), "wb") as f:
        pickle.dump(output, f)


def fig_SI_bif_data(params_A, params_B):

    output = {}
    for key, params in zip(["A", "B"], [params_A, params_B]):
        output[key] = bifurcation_data(params, compute_SI, 0.01, 0.01, 2.4, "SI", key)    

    with open(fn("SI_bif"), "wb") as f:
        pickle.dump(output, f)
    
def fig_B_bif_data(params_A, params_B):

    output = {}
    for key, params in zip(["A", "B"], [params_A, params_B]):
        output[key] = bifurcation_data(params, compute_B, 0.1, 0.01, 2.0, "B", key)    
              

    with open(fn("B_bif"), "wb") as f:
        pickle.dump(output, f)
    
def fig_F_bif_data(params_A, params_B):
    output = {}
    for key, params in zip(["A", "B"], [params_A, params_B]):
        output[key] = bifurcation_data(params, compute_F, -1.5, 0.01, 1.5, "F", key)    

    with open(fn("F_bif"), "wb") as f:
        pickle.dump(output, f)
    
        
def fig_gk_bif_data(params_B):
    output = {}
    for key, params in zip(["B"], [params_B]):
        output[key] = bifurcation_data(params, compute_gk, 0.1, 1, 3000, "gk", key)    
        

    with open(fn("gk_bif"), "wb") as f:
        pickle.dump(output, f)
       



In [ ]:
%%time 
params_A = dict(**common_params, **uncommon_params["A"], k_d=1e-3)    
params_A['β'] = params_A['beta_tot']  / params_A['γ']
params_B = dict(**common_params, **uncommon_params["B"], k_d=1e-3)
params_B['β'] = params_B['beta_tot']  / params_B['γ']
params_C = dict(**common_params, **uncommon_params["C"], k_d=1e-3)
params_C['β'] = params_C['beta_tot']  / params_C['γ']
params_D = dict(**common_params, **uncommon_params["C"], k_d=7*1e-2)
params_D['β'] = params_D['beta_tot']  / params_D['γ']

sp_default = dict(pre_sugar_days=14, sugar_days=28, post_sugar_days=7, insulin_days=28, post_insulin_days=140,low_sugar=small_meal_rate, high_sugar=large_meal_rate, insulin=insulin_rate)
sim_params = {}
for key, extra_insulin_days in zip(["A", "B","C","D"], [-28, 0, 28, 56]):
    sim_params[key] = dict(sp_default,insulin_days=sp_default["insulin_days"]+extra_insulin_days, post_insulin_days=sp_default["post_insulin_days"]-extra_insulin_days)

fig_2_data(params_A, params_B, params_C, params_D, sim_params["A"], sim_params["B"], sim_params["C"], sim_params["D"])

output = fig_3_data(params_A, params_B)
fig_4_data(params_A, params_B, insulin_rate)
blah = fig_5_data(params_A, params_B)
fig_6_data(params_B, sim_params["B"], 90)
fig_M_bif_data(params_A, params_B)
fig_SI_bif_data(params_A, params_B)
fig_B_bif_data(params_A, params_B)
fig_F_bif_data(params_A, params_B)
fig_gk_bif_data(params_B)



In [ ]:
#check rate of β-cell decline at 150 mg/dL glucose, as mentioned in discussion
models.evaluate(models.hill(params_B["g_K"], params_B["g_n"], params_B["g_k"]*params_B["k_r"]), 150.0)

In [ ]:
#estimate grams of sugar produced per day if rate = 2 mg/(kg min), then estimate corresponding rate in units of mg/(dL day)
#for discussion in methods section
rate = 2
body_mass = 70
minutes_in_day = 24*60
grams_per_mg = 0.001
print("rough amount of sugar produced per day (grams):",rate*body_mass*minutes_in_day*grams_per_mg)

body_plasma = 35 #3.5 L, in dL
print("rough HGP rate", rate*body_mass*minutes_in_day/body_plasma)

